# Profitable App to Launch based on Data Analysis

Our aim in this project is to identify competitive mobile device profiles for both the App Store and Google Play markets. Our job is to make data-driven decisions about the kind of applications that a team of developers need to create.

We are focussing on free apps, which generate revenue from in-app ads. This means that our income for any particular app depends mainly on the number of users using the application. Our mission is to evaluate data and help our developers recognize what sort of apps more consumers are likely to use. 

## Opening and Exploring Data

There are approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play.

Collecting data for over four million apps requires a significant amount of time and money, so we'll try to analyze a sample of data instead. To avoid spending resources with collecting new data ourselves, we should first try to see whether we can find any relevant existing data at no cost. Luckily, these are two data sets that seem suitable for our purpose:

* A data set containing data about approximately ten thousand Android apps from Google Play. You can download the data set directly from this [link](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv).
* A data set containing data about approximately seven thousand iOS apps from the App Store. You can download the data set directly from this [link](https://dq-content.s3.amazonaws.com/350/AppleStore.csv).

Let's start by opening the two datasets and then continue exploring the data.

In [ ]:
from csv import reader

#Creating a function which converts csv files to datasets of lists.
def csv_to_list(a_file, head = True):
    opened_file = open(a_file, encoding = 'utf-8')
    read_file = reader(opened_file)
    dataset = list(read_file)
    header = dataset[0]
    dataset = dataset[1:]
    return dataset, header

#Importing Datasets and converting them
gplay, gplay_header = csv_to_list('../input/google-and-apple-store/googleplaystore.csv')
appstore, appstore_header = csv_to_list('../input/google-and-apple-store/AppleStore.csv')

To make it easier to explore the two data sets, we'll first write a function named explore_data() that we can use repeatedly to explore rows in a more readable way. We'll also add an option for our function to show the number of rows and columns for any data set.

In [ ]:
def explore_data(dataset, start, end, rows_and_column = False):
    sliced_data = dataset[start:end]
    for x in sliced_data:
        print(x)
        print('\n')
    if rows_and_column:
        print('Number of rows are - ', len(dataset))
        print('Number of columns are - ', len(dataset[0]))
        
print(gplay_header)
print('\n')
explore_data(gplay, 0, 3, True)

We see that the Google Play data set has 10841 apps and 13 columns. At a quick glance, the columns that might be useful for the purpose of our analysis are 'App', 'Category', 'Reviews', 'Installs', 'Type', 'Price', and 'Genres'.

Now let's take a look at the App Store data set.

In [ ]:
print(appstore_header)
print('\n')
explore_data(appstore, 0, 3, True)

We have 7197 iOS apps in this data set, and the columns that seem interesting are: 'track_name', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', and 'prime_genre'.

# Cleaning Data

In [ ]:
for x in appstore:
    if x[5] == 'USD':
        print(x)

In [ ]:
del gplay[10472]

In [ ]:
def delete_row(a_list, term, column):
    temp_list = []
    for x in a_list:
        if x[column] != term:
            temp_list.append(x)
    return temp_list

### Checking Duplicate Apps

In [ ]:
duplicate_apps = []
unique_apps = []

for app in gplay:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
    
print('Number of duplicate apps:', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps:', duplicate_apps[:15])

In [ ]:
#Removing Multiple Entries
def remove_multiple(a_list, index):
   
    review_count = {}
    for x in a_list:
        name = x[0]
        if name not in review_count:
            review_count[name] = float(x[index])
        elif float(x[index]) > review_count[name]:
            review_count[name] = float(x[index])
    clean_list = []
    already_added = []
    for x in a_list:
        name = x[0]
        if (float(x[index]) == review_count[name]) and (name not in already_added):
            clean_list.append(x)
            already_added.append(name)
    
    return clean_list


            
gplay = remove_multiple(gplay, 3)
appstore = remove_multiple(appstore, 5)

### Filtering for English only Apps

English language's ASCII codes range from 0-127, if the character is out range, then it's most probably another language.

If we remove all the apps with single character exceeding the range, we might even skip the english apps which have emojis. Therefore, we will remove the apps which have more than 3 characters who are out of range.

In [ ]:
def is_english(a_list, index):
    temp_list = []
    for x in a_list:
        name = x[index]
        counter = 0
        for y in name:
            if ord(y) > 127:
                counter += 1
        if counter < 3:
            temp_list.append(x)
        counter = 0
    return temp_list

gplay = is_english(gplay, 0)
appstore = is_english(appstore, 1)

print(len(gplay))
print(len(appstore))

### Isolating Free Apps

In [ ]:
def free_apps(a_list, index):
    temp_list = []
    for x in a_list:
        price = x[index]
        if price == '0':
            temp_list.append(x)
        elif price == '0.0':
            temp_list.append(x)
    return temp_list    

gplay = free_apps(gplay, 7)
appstore = free_apps(appstore, 4)

print(len(gplay))
print(len(appstore))

___

# Analysing Data

As we stated in the introduction, our goal is to determine the types of apps that are likely to draw more customers, as our revenue is strongly affected by the amount of people using our applications.

Our validation approach for an product concept is comprised of three measures to reduce costs and overhead:
* Create and add a limited edition of the Android app to Google Play
* If the app has a strong consumer reaction, we'll further improve it
* If the product is successful after six months, we'll create and attach an iOS edition of the app to the App Store

Since our ultimate goal is to introduce the app to both Google Play and the App Store, we need to identify good user profiles in both markets. A profile that fits well for both sectors , for example, may be a productivity app that makes use of gamification.

Let's continue the analysis by having a sense of what the most popular genres are for any market. For this we would need to build frequency tables in our data sets for a few columns.

### Finding the most common genre

In [ ]:
#Making a fucntion which takes in a list and index and returns the percentages of the column.

def column_percentage(a_list, index):
    column_dict = {}
    for x in a_list:
        if x[index] in column_dict:
            column_dict[x[index]] += 1
        else:
            column_dict[x[index]] = 1
    
    #finding total entries
    total = 0
    for x in column_dict:
        total += column_dict[x]
    
    #Updating dictionary to percentage
    for x in column_dict:
        column_dict[x] = float((column_dict[x]/total)*100)
    
    return sort_dict(column_dict)
    
def sort_dict(a_dict): 
    table_display = []
    for x in a_dict:
        key_val = (a_dict[x], x)
        table_display.append(key_val)
    table_sorted = sorted(table_display, reverse = True)
    list_sorted = list(table_sorted)
    return list_sorted

**Appstore Most Common Genres**

Appstore dataset has the column 'prime_genre' and we analysed that below.

In [ ]:
genre_percent_appstore = column_percentage(appstore, -5)
explore_data(genre_percent_appstore, 0, 5)

We can see that more than a half (58.16 per cent) of the free English applications are games. Entertainment applications are approximately 8%, led by photo and video apps, close to 5%. Just 3.66% of the applications are built for education, accompanied by social networking apps that make up 3.29% of the apps in our data set.

The general impression is that the App Store (at least the section that includes free English content) is overrun by applications built for fun (games, entertainment, photo and video, social networking, sports , music, etc.) whereas practical applications (education, shopping, services, efficiency, lifestyle, etc.) are more uncommon. Although the fact that fun applications are the most popular still doesn't mean that they will have the highest amount of users — the demand may not be the same as the supply.

**Google Play Store Most Common Genres**

Google Play dataset has two similar columns, "Category" and "Genres". We will analyse both the datasets below.

In [ ]:
# Analysing the column "Category"

genre_percent_gplay = column_percentage(gplay, 1)
explore_data(genre_percent_gplay, 0, 5)

In [ ]:
# Analysing the column "Genres"

genre_percent_gplay = column_percentage(gplay, 9)
explore_data(genre_percent_gplay, 0, 5)

On Google Play, the environment appears dramatically different: there aren't too many applications built for entertainment, and a large number of applications seem to be developed for practical purposes (family, resources, company, lifestyle, efficiency, etc.). However, if we further examine this, we will see that the family group (which accounts for approximately 19 percent of the apps) mainly includes games for children.

Even so, it seems that realistic applications are best portrayed on Google Play relative to the App Store. The frequency table we see for the Genres column also supports this picture

The distinction between the columns of the Genres and the columns of the Category is not perfectly clear but one aspect we should note is that the column of Genres is far more granular (it has more categories). At the moment we 're just aiming at the larger picture and we're only going to deal by moving on in the Category column.

Up until this point , we found that applications made for fun rule the App Store, while Google Play presents a more diverse ecosystem of both realistic and non-fun devices. Now we would like to get an understanding about what kind of apps most users have.

---

### Finding the most popular genre on Appstore

One way to figure out which genres are most popular (have the most users) is by measuring the total amount of installs for each genre of apps. For the Google Play data set, we can find this information in the Installs column,  but this detail is lacking for the App Store data set.  As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the rating_count_tot app.

#### Finding the average number of user ratings per app genre on the App Store

To do that, we'll need to:
* Isolate the apps of each genre
* Sum up the user ratings for the apps of that genre
* Divide the sum by the number of apps belonging to that genre

In [ ]:
#Function to calculate the above

def avg_by_ratings(a_list, genre_index, rating_count_index):
    column_dict = {}
    for x in a_list:
        if x[genre_index] in column_dict:
            column_dict[x[genre_index]] += 1
        else:
            column_dict[x[genre_index]] = 1
    
    ratings_sum_dict = {}
    for x in a_list:
        if x[genre_index] in ratings_sum_dict:
            ratings_sum_dict[x[genre_index]] += float(x[rating_count_index])
        else:
            ratings_sum_dict[x[genre_index]] = float(x[rating_count_index])
    
    #finding total entries
    total = 0
    for x in column_dict:
        total += column_dict[x]        
    
    percentage_dict = {}
    for x in column_dict:
        percentage_dict[x] = (ratings_sum_dict[x]/column_dict[x])*100
    
    return sort_dict(percentage_dict)
    

In [ ]:
popular_appstore = avg_by_ratings(appstore, -5, 5)
explore_data(popular_appstore, 0, 5)

Navigation applications have the largest amount of user ratings on average, although this statistic is highly skewed by Waze and Google Maps, who have almost half a million user reviews combined

In [ ]:
for x in appstore:
    if x[-5] == "Navigation":
        print(x[1], ':', x[5])

The same trend extends to social networking apps, where the total amount is highly skewed by a few giants such as Twitter , Instagram, Skype etc. The same happens to music applications, where the total figure is highly skewed by a few major players like Itunes , Spotify and Shazam.

Reference applications have an overall consumer ranking of 74,942, but in reality it is the Bible and Dictionary.com that have inflated the total rating:

In [ ]:
for x in appstore:
    if x[-5] == "Reference":
        print(x[1], ':', x[5])

The niche seems to be showing some promise, however;  one thing we can do is take another famous book and convert it into an app where besides the raw book edition, we should incorporate various apps. These may include regular book quotations, an audible recording of the book, book quizzes etc. We might even integrate a dictionary into the app, meaning people don't have to quit our app to look up terms in an unrelated site.

The concept seems to suit well with the reality that for-fun applications rule the App Store. It means that the competition might be a little crowded with for-fun apps, suggesting that an useful app would have the potential to stick out from the large amount of applications on the App Store.

Certain categories that tend to be common include weather, book, food and drink, or finance. The genre of the book seems to correlate a bit with the theme of the app that we mentioned above, but the other genres do not seem that important to us:
* Weather applications — users usually don't spend that much time in the app, so there's no chance of profiting from in-app adds. It could also require us to link our apps to non-free APIs to get accurate live weather data.
* Food and drink - Starbucks, Dunkin' Donuts, McDonald's, etc. are examples here. So creating a famous food and drink app needs true cooking and distribution operation, which is outside our company's reach.
* Apps for finance — such features provide insurance, bill paying, money transfer, etc. Creating a finance app calls for awareness of the subject and we don't want to employ a finance specialist just to create an app.

Now let's look a little more at the demand for Google Play.

### Finding the most popular genre on Google Play

We already have details on the amount of downloads for the Google Play market and we will be able to get a better view of the success of the genre. Nonetheless, the number of installations does not seem specific enough — we can see that the bulk of numbers are open-ended (100 +, 1,000 +, 5,000 +, etc.)

In [ ]:
def find_unique(a_list, index):
    installs_dict = {}
    for x in a_list:
        if x[index] in installs_dict:
            installs_dict[x[index]] += 1
        else:
            installs_dict[x[index]] = 1
            
    return sort_dict(installs_dict)

gplay_installs = find_unique(gplay, 5)
explore_data(gplay_installs, 0, 5)

The issue with the data is that it isn't precise. For example, we don't know whether an app that has 100,000 + installs has 100,000, 200,000, or 350,000 installs. Nonetheless, for our purposes, we don't require really accurate numbers — we only want to get an understanding of which device categories are drawing the most consumers, so we don't need absolute accuracy in terms of number of consumers.

We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.



To perform computations, however, we'll need to convert each install number to float — this means that we need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error. We'll do this directly in the loop below, where we also compute the average number of installs for each genre (category).

In [ ]:
# Function to remove commas and plus, and find avg
def make_number(a_list, index, index_c):
    temp1_list = []
    for x in a_list:
        temp1_list.append(x)
    temp_list = []
    for x in temp1_list:
        installs = str(x[index])
        installs = installs.replace(',','')
        installs = installs.replace('+','')
        installs = installs.replace(' ','')
        x[index] = float(installs)
        temp_list.append(x)
    
    freq_installs = {}
    genre_count = {}
    total_installs = 0
    for x in temp_list:
        if x[index_c] in freq_installs:
            freq_installs[x[index_c]] += x[index]
            genre_count[x[index_c]] += 1
        else:
            freq_installs[x[index_c]] = x[index]
            genre_count[x[index_c]] = 1
        total_installs += x[index]

    avg_installs = {}
    for x in freq_installs:
        avg_installs[x] = freq_installs[x] / genre_count[x]
    
    return sort_dict(avg_installs)

gplay_avg = make_number(gplay, 5, 1)
explore_data(gplay_avg, 0, 10)

Communication applications have the most installs, on average: 38,590,581. This figure is severely skewed by a few devices with over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with more than 100 million and 500 million installs:

In [ ]:
for x in gplay:
    if x[1] == 'COMMUNICATION' and (x[5] == 1000000000 or x[5] == 500000000 or x[5] == 100000000):
        print(x[0], ':', x[5],'+')

For the category of video players we see the same trend which is the runner-up with 24,727,872 installs. Apps such as the Youtube , Google Play Movies & TV, or MX Player control the business. For social applications (where we have giants like Twitter, Snapchat, Google+, etc.), photography apps (Google Images and other common picture editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.), the trend is replicated. 

The big problem is again that these types of apps may seem more common than they actually are. In fact, these niches tend to be occupied by a few giants that are impossible to compete with.

The game genre seems fairly common, but we've pointed out previously that this section of the market seems a little crowded, so if possible we 'd like to come up with a specific app suggestion. 

The books and genre of comparison also seems relatively common, with an estimated number of 8,767,811 installs. Exploring this in more detail is fascinating, because we find this genre has some potential to perform well on the App Store, and our intention is to suggest an device genre that demonstrates potential to be competitive on both the App Store and Google Play as well.

Let's look at some of the genre's applications and their amount of installations:

In [ ]:
for x in gplay:
    if x[1] == 'BOOKS_AND_REFERENCE':
        print(x[0], ':', x[5])

The book and reference category covers a range of applications: ebook editing and reading tools, different libraries, dictionaries, computer or language tutorials, etc. There also appears to be a limited range of extremely popular apps that were skewing the average:

In [ ]:
for x in gplay:
    if x[1] == 'BOOKS_AND_REFERENCE' and (x[5] == 1000000
                                            or x[5] == 5000000
                                            or x[5] == 10000000
                                            or x[5] == 50000000):
        print(x[0], ':', x[5],'+')

This market appears to be filled by ebook scanning and reading applications, as well as numerous libraries and dictionaries, and creating related apps definitely isn't a smart idea because there would be some serious rivalry. 

We also note that there are quite a few apps developed around the Quran book and indicate that it could be lucrative to develop an app around a common book. This seems that having a famous book (maybe a more recent novel) and transforming it into an app might be beneficial for both the industries of Google Play and the App Store.

It seems like the market is still full of libraries, though, and we ought to incorporate any extra features apart from the book's original edition. These may include frequent quotations from the novel, an audible edition of the novel, book quizzes, a website where people will chat about the book etc.

## Conclusions

In this project , we analyzed data on smartphone applications from the App Store and Google Play with the goal of suggesting an app profile that could be competitive for both markets. 

We decided that it would be beneficial for both the Google Play and the App Store markets to take a famous book (perhaps a more recent novel) and convert it into an app. The markets are now full of libraries and in addition to the raw edition of the novel, we need to introduce certain unique features. These may include frequent quotations from the novel, an audible edition of the novel, book quizzes, a website where people will chat about the book etc.